In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("__file__"), "src")))

In [27]:
from bot.web_interface import WebInterface
from bot.chess_engine import ChessEngine
from bot.chess_bot import ChessBot

In [28]:
# Example setup
stockfish_path = (
    "../stockfish-ubuntu-x86-64-bmi2/stockfish/stockfish-ubuntu-x86-64-bmi2"
)
engine = ChessEngine(stockfish_path)
web_interface = WebInterface()  # assuming you have a browser instance

In [34]:
chess_bot = ChessBot(web_interface, engine)
# Play a game
chess_bot.play_game(
    playing_as="white", mode="debug", move_speed="bullet", timeout=15, loop_speed=0.1
)

Best move: e2e4
Performing move: e2 to e4
{'top_left': (268.0, 402.0), 'bottom_right': (335.0, 469.0)}
Dragged from e2 to e4 in 0.0861043930053711 seconds.
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
New computer move detected: c5
Detected computer's move: c5
Evaluation after computer's move: {'type': 

AttributeError: 'Stockfish' object has no attribute 'send_quit_command'

In [10]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import os
import undetected_chromedriver as uc
import time
import re

In [11]:
web_interface.browser_is_running

True

In [19]:
promotion_match = re.match(r"([a-h][1-8])([a-h][1-8])(q|r|b|n)", "b7a8r")
if promotion_match:
    start_square, end_square, promotion_piece = promotion_match.groups()
    print(
        f"Start square: {start_square}, End square: {end_square}, Promotion piece: {promotion_piece}"
    )
else:
    print("No promotion detected")

Start square: b7, End square: a8, Promotion piece: r


In [18]:
promotion_match = re.match(r"([a-h][18])=(q|n|r|b)([+#]?)", "b7a8r")
promotion_square, promotion_piece, check_or_mate = promotion_match.groups()
promotion_square

AttributeError: 'NoneType' object has no attribute 'groups'

In [21]:
handle_pawn_promotion("a7a8r")

Start square: a7, End square: a8, Promotion piece: r
Clicked on promotion piece: rook


In [20]:
def handle_pawn_promotion(move_san):
    """
    Detects if the move is a pawn promotion (e.g., g8=Q), and selects the correct
    promotion piece in the UI by clicking the appropriate element.

    Parameters:
    - browser: The Selenium WebDriver instance.
    - move_san: The SAN string of the move, e.g., 'g8=Q' for a promotion to queen.
    """
    promotion_match = re.match(r"([a-h][1-8])([a-h][1-8])(q|r|b|n)", move_san)

    if promotion_match:
        start_square, end_square, promotion_piece = promotion_match.groups()
        print(
            f"Start square: {start_square}, End square: {end_square}, Promotion piece: {promotion_piece}"
        )
        # Wait until the promotion choice UI is visible (with the correct ID)
        try:
            promotion_choice_element = WebDriverWait(web_interface.browser, 10).until(
                EC.visibility_of_element_located((By.ID, "promotion-choice"))
            )
        except TimeoutException:
            print("Promotion choice element not found.")
            return

        # Define a mapping of piece notation to the expected CSS class
        piece_class_mapping = {
            "q": "queen",
            "n": "knight",
            "r": "rook",
            "b": "bishop",
        }

        # Get the piece class to select (e.g., 'queen', 'knight', etc.)
        piece_class = piece_class_mapping.get(promotion_piece)

        if not piece_class:
            print(f"Invalid promotion piece: {promotion_piece}")
            return

        # Find the correct piece element to click within the promotion choice UI
        try:
            # The pieces are inside squares, so find the correct square
            promotion_squares = promotion_choice_element.find_elements(
                By.CSS_SELECTOR, "square"
            )

            for square in promotion_squares:
                piece = square.find_element(By.CSS_SELECTOR, "piece")
                piece_class_attribute = piece.get_attribute("class")

                if piece_class in piece_class_attribute:
                    # Found the correct promotion piece, now click on it
                    square.click()
                    print(f"Clicked on promotion piece: {piece_class}")
                    break
            else:
                print(f"Could not find the promotion piece: {promotion_piece}")
        except NoSuchElementException:
            print(f"Error: Could not find elements for promotion selection.")

    else:
        print("No promotion detected")
        return